In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score
import scipy.io
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df_nulos = pd.read_csv("data/data.csv")
df_original = pd.read_csv("data/Data_CHF_Zhao_2020_ATE.csv")

In [3]:
# Aplicar codificación one-hot a la columna "author"
df_nulos = pd.get_dummies(df_nulos, columns=['author'], prefix=['author'])

# Aplicar codificación one-hot a la columna "geometry"
df_nulos = pd.get_dummies(df_nulos, columns=['geometry'], prefix=['geometry'])

In [4]:
# Aplicar codificación one-hot a la columna "author"
df_original = pd.get_dummies(df_original, columns=['author'], prefix=['author'])

# Aplicar codificación one-hot a la columna "geometry"
df_original = pd.get_dummies(df_original, columns=['geometry'], prefix=['geometry'])

In [5]:
# Filtrar las filas con valores nulos en la columna 'x_e_out'
df_nulos1 = df_nulos[df_nulos['x_e_out [-]'].isnull()]

# Crear un nuevo DataFrame con las filas que contienen valores nulos en 'x_e_out'
nuevos_datos = df_nulos1.copy()

In [6]:
nuevos_datos

,id,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],author_Beus,author_Inasaka,...,author_Kossolapov,author_Mortimore,author_Peskov,author_Richenderfer,author_Thompson,author_Weatherhead,author_Williams,geometry_annulus,geometry_plate,geometry_tube
4,4,13.79,686.0,NaN,11.1,11.1,457.0,2.8,0,0,...,0,0,0,0,0,0,0,0,0,1
7,7,18.00,750.0,NaN,10.0,10.0,1650.0,2.2,0,0,...,0,0,1,0,0,0,0,0,0,1
10,10,NaN,NaN,NaN,1.9,1.9,152.0,3.2,0,0,...,0,0,0,0,1,0,0,0,0,1
12,12,6.89,7500.0,NaN,NaN,12.8,1930.0,4.8,0,0,...,0,0,0,0,1,0,0,0,0,0
23,23,15.51,1355.0,NaN,5.6,15.2,2134.0,2.1,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31633,31633,11.03,NaN,NaN,11.5,11.5,NaN,2.0,0,0,...,0,0,0,0,1,0,0,0,0,1
31634,31634,1.01,2000.0,NaN,15.0,120.0,10.0,6.2,0,0,...,0,0,0,1,0,0,0,0,1,0
31637,31637,13.79,688.0,NaN,NaN,11.1,457.0,2.3,0,0,...,0,0,0,0,0,1,0,0,0,1
31640,31640,13.79,NaN,NaN,4.7,4.7,NaN,3.9,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df_nulos = df_nulos.dropna(subset=['x_e_out [-]'])

# Imprimir el DataFrame original sin las filas con valores nulos



In [8]:
df_nulos

,id,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],author_Beus,author_Inasaka,...,author_Kossolapov,author_Mortimore,author_Peskov,author_Richenderfer,author_Thompson,author_Weatherhead,author_Williams,geometry_annulus,geometry_plate,geometry_tube
0,0,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6,0,0,...,0,0,0,0,1,0,0,0,0,1
1,1,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2,0,0,...,0,0,0,0,1,0,0,0,0,1
2,2,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5,0,0,...,0,0,0,0,1,0,0,0,0,0
3,3,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0,1,0,...,0,0,0,0,0,0,0,1,0,0
5,5,17.24,3648.0,-0.0711,NaN,1.9,696.0,3.6,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31636,31636,12.07,NaN,-0.0195,NaN,1.9,152.0,5.4,0,0,...,0,0,0,0,0,0,0,0,0,0
31638,31638,NaN,3648.0,-0.0487,4.7,4.7,318.0,9.0,0,0,...,0,0,0,0,1,0,0,0,0,1
31639,31639,NaN,1736.0,0.0886,NaN,7.8,591.0,2.3,0,0,...,0,0,0,0,1,0,0,0,0,0
31641,31641,18.27,658.0,-0.1224,3.0,3.0,150.0,2.3,0,0,...,0,0,0,0,1,0,0,0,0,0


In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense

# Separar las columnas de características (X) y el objetivo (y) en ambos conjuntos
X_df_original = df_original.drop("x_e_out [-]", axis=1)
y_df_original = df_original["x_e_out [-]"]

# Separar las columnas de características (X) y el objetivo (y) en ambos conjuntos
X_df_nulos = df_nulos.drop("x_e_out [-]", axis=1)
y_df_nulos = df_nulos["x_e_out [-]"]

# Escalar los datos
scaler = MinMaxScaler()
X_sin_nulos_scaled_original = scaler.fit_transform(X_df_original)
X_con_nulos_scaled = scaler.transform(X_df_nulos)

# Construir y entrenar el autoencoder
autoencoder = Sequential()
# Agregar capas al autoencoder (ajusta la estructura según tus necesidades)
autoencoder.add(Dense(64, activation="relu", input_shape=(X_df_original.shape[1],)))
autoencoder.add(Dense(32, activation="relu"))
autoencoder.add(Dense(64, activation="relu"))
autoencoder.add(Dense(X_df_original.shape[1], activation="sigmoid"))
autoencoder.compile(optimizer="adam", loss="mse")
autoencoder.fit(X_sin_nulos_scaled_original, X_sin_nulos_scaled_original, epochs=10, batch_size=32)

# Aplicar el autoencoder para rellenar los nulos en X_con_nulos
X_con_nulos_imputados_scaled = autoencoder.predict(X_con_nulos_scaled)
X_con_nulos_imputados = scaler.inverse_transform(X_con_nulos_imputados_scaled)

# Reemplazar los nulos en el dataframe con los valores imputados
df_con_nulos_imputados = df_nulos.copy()
df_con_nulos_imputados[X_df_nulos.columns] = X_con_nulos_imputados


Epoch 1/10
59/59 [==============================] - 1s 3ms/step - loss: 0.1247
Epoch 2/10
59/59 [==============================] - 0s 1ms/step - loss: 0.0278
Epoch 3/10
59/59 [==============================] - 0s 1ms/step - loss: 0.0145
Epoch 4/10
59/59 [==============================] - 0s 1ms/step - loss: 0.0096
Epoch 5/10
59/59 [==============================] - 0s 1ms/step - loss: 0.0065
Epoch 6/10
59/59 [==============================] - 0s 1ms/step - loss: 0.0049
Epoch 7/10
59/59 [==============================] - 0s 1ms/step - loss: 0.0041
Epoch 8/10
59/59 [==============================] - 0s 2ms/step - loss: 0.0036
Epoch 9/10
59/59 [==============================] - 0s 2ms/step - loss: 0.0033
Epoch 10/10
664/664 [==============================] - 1s 886us/step


In [27]:
df_con_nulos_imputados

,id,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],author_Beus,author_Inasaka,...,author_Kossolapov,author_Mortimore,author_Peskov,author_Richenderfer,author_Thompson,author_Weatherhead,author_Williams,geometry_annulus,geometry_plate,geometry_tube
0,NaN,NaN,NaN,0.1754,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,-0.0416,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,815.107727,14.049908,4694.448242,0.0335,10.804685,13.285516,1532.855469,5.857305,1.422520e-02,5.115336e-03,...,1.162548e-02,8.888188e-03,9.043359e-02,5.051452e-03,9.600036e-01,0.002279,5.820260e-03,4.133191e-02,0.003237,0.984192
3,1159.198486,13.332467,3946.221924,-0.0279,5.425097,13.550525,1895.304443,3.306559,9.460205e-01,1.114758e-03,...,4.265870e-03,1.147917e-02,4.932328e-02,6.785952e-03,6.493181e-05,0.007114,1.409051e-04,9.817193e-01,0.004377,0.021089
5,NaN,NaN,NaN,-0.0711,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31636,NaN,NaN,NaN,-0.0195,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31638,NaN,NaN,NaN,-0.0487,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31639,NaN,NaN,NaN,0.0886,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31641,1864.999878,0.164979,9.614677,-0.1224,1.002541,45.135036,10.000000,0.800039,3.468642e-11,1.575507e-18,...,1.047667e-08,2.017865e-18,5.425231e-20,5.694415e-13,8.258252e-24,0.000128,8.007540e-22,2.708035e-12,0.006962,0.918897


In [13]:
df_nulos

,id,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],author_Beus,author_Inasaka,...,author_Kossolapov,author_Mortimore,author_Peskov,author_Richenderfer,author_Thompson,author_Weatherhead,author_Williams,geometry_annulus,geometry_plate,geometry_tube
0,0,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6,0,0,...,0,0,0,0,1,0,0,0,0,1
1,1,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2,0,0,...,0,0,0,0,1,0,0,0,0,1
2,2,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5,0,0,...,0,0,0,0,1,0,0,0,0,0
3,3,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0,1,0,...,0,0,0,0,0,0,0,1,0,0
5,5,17.24,3648.0,-0.0711,NaN,1.9,696.0,3.6,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31636,31636,12.07,NaN,-0.0195,NaN,1.9,152.0,5.4,0,0,...,0,0,0,0,0,0,0,0,0,0
31638,31638,NaN,3648.0,-0.0487,4.7,4.7,318.0,9.0,0,0,...,0,0,0,0,1,0,0,0,0,1
31639,31639,NaN,1736.0,0.0886,NaN,7.8,591.0,2.3,0,0,...,0,0,0,0,1,0,0,0,0,0
31641,31641,18.27,658.0,-0.1224,3.0,3.0,150.0,2.3,0,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

# Paso 1: Dividir los datos originales en X_train y X_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 2: Cargar el otro conjunto de datos
df_additional = pd.read_csv('additional_data.csv')

# Paso 3: Dividir el otro conjunto de datos en X_additional y y_additional
X_additional = df_additional.drop('target', axis=1)
y_additional = df_additional['target']

# Paso 4: Concatenar X_additional y y_additional a X_train y y_train
X_train = np.concatenate((X_train, X_additional), axis=0)
y_train = np.concatenate((y_train, y_additional), axis=0)

# Paso 5: Definir las transformaciones y el modelo
imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()
model = XGBRegressor()

# Paso 6: Crear el pipeline
pipeline = Pipeline([
    ('imputer', imputer),
    ('scaler', scaler),
    ('model', model)
])

# Paso 7: Ajustar el pipeline a los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Paso 8: Realizar predicciones en los datos de prueba
y_pred = pipeline.predict(X_test)